In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# libraries
import riiideducation
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from typing import List, Dict, Optional
import numpy as np
from sklearn.model_selection import RepeatedKFold
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import time
import random
import lightgbm as lgb
import gc
import os
from sklearn.preprocessing import LabelEncoder
from numba import jit
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline
plt.rcParams['figure.figsize']=10,6
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    

@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True

In [ ]:
# Genersting the pie chart
def pie_chart(df,col,path):
  label = df[col].value_counts().index.tolist()
  fig = plt.figure(figsize=(10,6))
  ax = (df[col].value_counts()*100.0 /len(df))\
  .plot.pie(startangle=90,autopct='%.1f%%', labels =label, fontsize=12)                                                                           
  ax.set_title('% '+str(col))
  # plt.savefig(path+str(col1)+'.png')
  plt.show()
    
def bar_chart(df,parameter, figsize=(10,6)):
    target_counts = df[parameter].value_counts()
    target_perc = target_counts.div(target_counts.sum(), axis=0)
    plt.figure(figsize=figsize)
    ax = sns.barplot(x=target_counts.index.values, y=target_counts.values, order=target_counts.index,palette='winter')
#     plt.xticks(rotation=90)
    plt.xlabel(f'{parameter}', weight ='bold',fontsize=16)
    plt.ylabel('# of occurances', weight ='bold',fontsize=16)
    plt.title("Count of "+f'{parameter}', weight ='bold',fontsize=20)

    rects = ax.patches
    labels = np.round(target_perc.values*100, 2)
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, f'{label}%', ha='center', va='bottom')
    
    try:
        labels =target_counts.index.tolist()

    #     labels.sort()
        labels=[textwrap.fill(text,12) for text in labels]
        pos = np.arange(len(labels)) 
        plt.xticks(pos, labels,fontsize=12)
    except:
        pass

In [ ]:
path = '/kaggle/input'

train = pd.read_csv(f'{path}/riiid-test-answer-prediction/train.csv', low_memory=False, nrows=5 * (10**5),
                    usecols=['timestamp', 'user_id', 'content_id', 'content_type_id', 'user_answer', 'answered_correctly',
                             'prior_question_elapsed_time', 'prior_question_had_explanation'],
                       dtype={'timestamp': 'int64',
                              'user_id': 'int32',
                              'content_id': 'int16',
                              'content_type_id': 'int8',
                              'user_answer': 'int8',
                              'answered_correctly': 'int8',
                              'prior_question_elapsed_time': 'float32', 
                              'prior_question_had_explanation': 'boolean',
                             }
                      )
train = train.sort_values(['timestamp'], ascending=True)
questions = pd.read_csv(f'{path}/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv(f'{path}/riiid-test-answer-prediction/lectures.csv')
print('Train shapes: ', train.shape)     

In [ ]:
train.head()

In [ ]:
questions.head()

In [ ]:
lectures.head()

In [ ]:
# Check the cardinality of each columns
cardinality_df = pd.DataFrame()
l = []
for i in range(len(train.columns)):
#     print(i)
    distinct_count = train[train.columns[i]].value_counts().count()
    l.append(distinct_count)
col_name = train.columns.tolist()
cardinality_df["Columns_Name"]= col_name
cardinality_df['Cardinality_Count'] = l
cardinality_df

In [ ]:
sns.distplot(train.prior_question_elapsed_time)
plt.show()

In [ ]:
bar_chart(train,'content_type_id')

In [ ]:
bar_chart(train,'user_answer')

In [ ]:
bar_chart(train,'answered_correctly')

In [ ]:
bar_chart(train,'prior_question_had_explanation')

In [ ]:
gr_df=train.groupby(['content_type_id','answered_correctly'])['user_id'].count().to_frame().rename(columns={'user_id':'count'}).reset_index()

In [ ]:
g = sns.catplot(x="content_type_id", y="count",col="answered_correctly",
                data=gr_df, kind="bar",
                height=6, aspect=.7);

In [ ]:
dgen_df = train.groupby(['user_answer'])['answered_correctly'].count().to_frame().reset_index()

In [ ]:
sns.catplot(x="user_answer", y="answered_correctly",
                data=dgen_df, kind="bar");

In [ ]:
rv_df = train.groupby(['user_answer','answered_correctly'])['user_id'].count().to_frame().rename(columns={'user_id':'count'}).reset_index()

In [ ]:
g = sns.catplot(x="user_answer", y="count",col="answered_correctly",
                data=rv_df, kind="bar",
                height=6, aspect=.9);

In [ ]:
def description(df):
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary

In [ ]:
description(train)

## Distribution of Target Variable

In [ ]:
pie_chart(train,'answered_correctly',8)

In [ ]:
train.columns

In [ ]:
id_col = ['user_id', 'content_id', 'content_type_id', 'prior_question_elapsed_time']
plt.figure(figsize=(10,6))
for i, col in enumerate(id_col):
    plt.subplot(2, 2, i + 1)
    sns.distplot(train[col], color='green', 
                 hist_kws={'alpha':1,"linewidth": 2},
                 kde_kws={"color": "red", "lw": 2, 'bw':0.01})
    plt.title(col)
    plt.tight_layout()

In [ ]:
time_col = ['timestamp', 'prior_question_elapsed_time',]
plt.figure(figsize=(10,6))
for i, col in enumerate(time_col):
    plt.subplot(1, 2, i + 1)
    train[col].hist(bins = 50,color='red')
    plt.title(col)
    plt.tight_layout()

> Timestamp represents the time from the first user interaction to the current one and Prior question elapsed time represents how long it took a user to answer their previous question bundle.

In [ ]:
col = ['prior_question_had_explanation', 'user_answer',]

total = len(train)
plt.figure(figsize=(12,5), dpi=60)

for i, col in enumerate(col):
    plt.subplot(1, 2, i + 1)
    g=sns.countplot(train[col], palette='winter')
    sizes=[] # Get highest values in y
    for p in g.patches:
        height = p.get_height()
        sizes.append(height)
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center", fontsize=14) 
    g.set_ylim(0, max(sizes) * 1.15) # set y limit based on highest heights
    plt.title(col)
    plt.tight_layout()

In [ ]:
# thanks to @ilialar 
train_only_df = train[train['answered_correctly']!=-1]
grouped_by_user_df = train_only_df.groupby('user_id')
user_answers_df = grouped_by_user_df.agg({'answered_correctly': ['mean', 'count'] })

fig,ax=plt.subplots(figsize=(15,8), dpi=100)

plt.subplot(2, 2, 1)
g1=user_answers_df[('answered_correctly','mean')].hist(bins=100, color='blue')
g1.set_title("users correct answer mean dist.",)

plt.subplot(2, 2, 2)
g2=user_answers_df[('answered_correctly','count')].hist(bins=100, color='blue')
g2.set_title('users correct answer count dist.')

plt.subplot(2, 2, 3)
g3=user_answers_df[user_answers_df[('answered_correctly','count')]<= 100][('answered_correctly','mean')].hist(bins=100, color='blue')
g3.set_title('users correct answer mean dist. less than 100 question')

plt.subplot(2, 2, 4)
g4=user_answers_df[user_answers_df[('answered_correctly','count')]>=100][('answered_correctly','mean')].hist(bins=100, color='blue')
g4.set_title('users correct answer count dist. more than 100 question')
plt.tight_layout()
plt.show()

> Average user score is lower than the overall % of correct answers(bottom left graph). It means heavy users have even better scores(bottom right graph).

In [ ]:
user_time_answers_df = grouped_by_user_df.agg({'answered_correctly': ['mean', 'count'],
                                        'timestamp': ['mean', 'count']})

fig,ax=plt.subplots(figsize=(15,6), dpi=50)

plt.subplot(1, 2, 1)
plt.scatter(x = user_answers_df[('answered_correctly','count')], 
            y = user_answers_df[ ('answered_correctly','mean')], color='green')
plt.title('relation b/w correct answer mean and count')

plt.subplot(1, 2, 2)
plt.scatter(x = user_time_answers_df[ ('timestamp','mean')], 
            y = user_time_answers_df[ ('answered_correctly','mean')], color='green')
plt.title('relation b/w  timestamp mean and correct answer mean')
plt.tight_layout()
plt.show()

> There is relationship between the average score for the active user, and the number of questions answered; there is relation average timestamp and average correct answer can be useful for baseline.

### Correct Answers by Content

In [ ]:
grouped_by_content_df = train_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg({'answered_correctly': ['mean', 'count'] })

fig,ax=plt.subplots(figsize=(15,8), dpi=100)

plt.subplot(2, 2, 1)
g1=content_answers_df[('answered_correctly','mean')].hist(bins=100, color='red')
g1.set_title("content correct answer mean dist.")

plt.subplot(2, 2, 2)
g2=content_answers_df[('answered_correctly','count')].hist(bins=100, color='red')
g2.set_title('content answer count dist.')

plt.subplot(2, 2, 3)
g3=content_answers_df[content_answers_df[('answered_correctly','count')]<= 100][('answered_correctly','mean')].hist(bins=100, color='red')
g3.set_title('content correct answer mean dist. less than 100 question')

plt.subplot(2, 2, 4)
g4=content_answers_df[content_answers_df[('answered_correctly','count')]>=100][('answered_correctly','mean')].hist(bins=100, color='red')
g4.set_title('content correct answer count dist. more than 100 question')
plt.tight_layout()
plt.show()

In [ ]:
user_time_content_df = grouped_by_content_df.agg({'answered_correctly': ['mean', 'count'],
                                        'timestamp': ['mean', 'count']})

fig,ax=plt.subplots(figsize=(15,6), dpi=70)

plt.subplot(1, 2, 1)
plt.scatter(x = content_answers_df[('answered_correctly','count')], 
            y=content_answers_df[ ('answered_correctly','mean')], color='cyan')
plt.title('relation b/w correct answer mean and count')

plt.subplot(1, 2, 2)
plt.scatter(x = user_time_content_df[ ('timestamp','mean')], 
            y = user_time_content_df[ ('answered_correctly','mean')], color='cyan')
plt.title('relation b/w  timestamp mean and correct answer count')
plt.tight_layout()
plt.show()

## Question.csv

In [ ]:
description(questions)

In [ ]:
id_col = ['question_id', 'bundle_id']
plt.figure(figsize=(10,6))
for i, col in enumerate(id_col):
    plt.subplot(1, 2, i + 1)
    sns.distplot(questions[col], color='green',bins=100, 
                 hist_kws={'alpha':1,"linewidth": 1},
                 kde_kws={"color": "red", "lw": 2, 'bw':0.01})
    plt.title(col)
    plt.tight_layout()

In [ ]:
questions_df = questions.merge(content_answers_df, left_on = 'question_id', right_on = 'content_id', how = 'left')
bundle_dict = questions_df['bundle_id'].value_counts().to_dict()

questions_df['right_answers'] = questions_df[('answered_correctly', 'mean')] * questions_df[('answered_correctly', 'count')]
questions_df['bundle_size'] = questions_df['bundle_id'].apply(lambda x: bundle_dict[x])

In [ ]:
questions_df.head()

In [ ]:
col = ['correct_answer', 'part', 'bundle_size']

total = len(questions_df)
plt.figure(figsize=(15,8), dpi=100)

for i, col in enumerate(col):
    plt.subplot(2, 2, i + 1)
    g=sns.countplot(questions_df[col], palette='winter')
    sizes=[] # Get highest values in y
    for p in g.patches:
        height = p.get_height()
        sizes.append(height)
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center", fontsize=14) 
    g.set_ylim(0, max(sizes) * 1.15) # set y limit based on highest heights
    plt.title(col)
    plt.tight_layout()

In [ ]:
fig,ax =plt.subplots(figsize=(15,6),dpi=70)
plt.subplot(1,2,1)
plt.scatter(x=questions_df['answered_correctly','count'],
           y=questions_df['right_answers'],color ='blue')
plt.title("relation b/w right answer and question asked (count)")

plt.subplot(1,2,2)
plt.scatter(x=questions_df['right_answers'],
           y=questions_df[('answered_correctly','mean')],color='blue')
plt.title("relation b/w right_answer and correct answer mean")
plt.tight_layout()
plt.show()

In [ ]:
grouped_by_bundle_df = questions_df.groupby('bundle_id')
bundle_answers_df = grouped_by_bundle_df.agg({'right_answers': 'sum', ('answered_correctly', 'count'): 'sum'}).copy()
bundle_answers_df.columns = ['bundle_rignt_answers', 'bundle_questions_asked']
bundle_answers_df['bundle_accuracy'] = bundle_answers_df['bundle_rignt_answers'] / bundle_answers_df['bundle_questions_asked']

In [ ]:
bundle_answers_df.head()

In [ ]:
fig,ax=plt.subplots(figsize=(15,6), dpi=100)
plt.subplot(1, 2, 1)
plt.scatter(x = bundle_answers_df['bundle_questions_asked'], 
            y=bundle_answers_df['bundle_accuracy'], color='dodgerblue')
plt.title('relation b/w bundle_questions_asked and bundle_accuracy')

plt.subplot(1, 2, 2)
plt.scatter(x = bundle_answers_df['bundle_rignt_answers'], 
            y = bundle_answers_df['bundle_accuracy'], color='dodgerblue')
plt.title('relation b/w  bundle_rignt_answers and bundle_accuracy')
plt.tight_layout()
plt.show()

In [ ]:
grouped_by_part_df = questions_df.groupby('part')
part_answers_df = grouped_by_part_df.agg({'right_answers': 'sum', ('answered_correctly', 'count'): 'sum'}).copy()
part_answers_df.columns = ['part_rignt_answers', 'part_questions_asked']
part_answers_df['part_accuracy'] = part_answers_df['part_rignt_answers'] / part_answers_df['part_questions_asked']
part_answers_df

In [ ]:
# del train_df
# del features_part_df
del questions_df
del bundle_answers_df
del grouped_by_part_df
del part_answers_df
del grouped_by_user_df
del grouped_by_content_df

In [ ]:
import gc
gc.collect()

In [ ]:
path = '/kaggle/input'

train = pd.read_csv(f'{path}/riiid-test-answer-prediction/train.csv',
                    usecols=['timestamp', 'user_id', 'content_id', 'content_type_id', 'user_answer', 'answered_correctly',
                             'prior_question_elapsed_time', 'prior_question_had_explanation'],
                       dtype={'timestamp': 'int64',
                              'user_id': 'int32',
                              'content_id': 'int16',
                              'content_type_id': 'int8',
                              'user_answer': 'int8',
                              'answered_correctly': 'int8',
                              'prior_question_elapsed_time': 'float32', 
                              'prior_question_had_explanation': 'boolean',
                             }
                      )
train = train.sort_values(['timestamp'], ascending=True)
questions = pd.read_csv(f'{path}/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv(f'{path}/riiid-test-answer-prediction/lectures.csv')
print('Train shapes: ', train.shape)

In [ ]:
# filter out lectures
train = train.loc[train['answered_correctly'] != -1].reset_index(drop=True)
train = train.drop(['timestamp','content_type_id'], axis=1)
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].fillna(value = False).astype(bool)

In [ ]:
user_answers_df = train.groupby('user_id').agg({'answered_correctly': ['mean', 'count']}).copy()
user_answers_df.columns = ['mean_user_accuracy', 'questions_answered']

content_answers_df = train.groupby('content_id').agg({'answered_correctly': ['mean', 'count']}).copy()
content_answers_df.columns = ['mean_accuracy', 'question_asked']

user_content_answers_df = train.groupby(['user_id', 'content_id']).agg({'answered_correctly': ['mean', 'count']}).copy()
user_content_answers_df.columns = ['mean_user_content_accuracy', 'content_questions_answered']

In [ ]:
#Now we will use only a part of data for training, to avoid leaks and memory error

train = train.iloc[90000000:,:]

In [ ]:
train = train.merge(user_answers_df, how = 'left', on = 'user_id')
train = train.merge(content_answers_df, how = 'left', on = 'content_id')
train = train.merge(user_content_answers_df, how = 'left', on = ['user_id', 'content_id'])

In [ ]:
train.fillna(value = 0.5, inplace = True)

In [ ]:
train['mean_diff1'] = train['mean_user_accuracy'] - train['mean_user_content_accuracy']
train['mean_diff2'] = train['mean_accuracy'] - train['mean_user_content_accuracy']

In [ ]:
train.head()

In [ ]:
le = LabelEncoder()
train["prior_question_had_explanation"] = le.fit_transform(train["prior_question_had_explanation"])

In [ ]:
train = train.sort_values(['user_id'])

In [ ]:
y = train['answered_correctly']

columns = ['mean_user_accuracy', 'questions_answered', 'mean_accuracy', 'question_asked',
           'prior_question_had_explanation', 'mean_diff1', 'mean_diff2', 'mean_user_content_accuracy']
X = train[columns]

In [ ]:
del train

In [ ]:
scores = []
feature_importance = pd.DataFrame()
models = []

In [ ]:
params = {'num_leaves': 32,
          'max_bin': 300,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "metric": 'auc',
         }

In [ ]:
columns = ['mean_user_accuracy', 'questions_answered', 'mean_accuracy', 'question_asked',
#            'prior_question_had_explanation', 'mean_diff1', 'mean_diff2'
          ]

In [ ]:
folds = StratifiedKFold(n_splits=5, shuffle=False)
for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
    print(f'Fold {fold_n} started at {time.ctime()}')
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    model = lgb.LGBMClassifier(**params, n_estimators=700, n_jobs = 1)
    model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=eval_auc,
            verbose=1000, early_stopping_rounds=10)
    score = max(model.evals_result_['valid_1']['auc'])
    
    models.append(model)
    scores.append(score)

    fold_importance = pd.DataFrame()
    fold_importance["feature"] = columns
    fold_importance["importance"] = model.feature_importances_
    fold_importance["fold"] = fold_n + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    break

In [ ]:
print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))


In [ ]:
feature_importance["importance"] /= 1
cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
    by="importance", ascending=False)[:50].index

best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

plt.figure(figsize=(16, 12));
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
plt.title('LGB Features (avg over folds)');

In [ ]:
del X, y

## Making predictions.
Code is taken from https://www.kaggle.com/sishihara/riiid-lgbm-5cv-benchmark

In [ ]:
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    y_preds = []
    test_df = test_df.merge(user_answers_df, how = 'left', on = 'user_id')
    test_df = test_df.merge(content_answers_df, how = 'left', on = 'content_id')
    test_df = test_df.merge(user_content_answers_df, how = 'left', on = ['user_id', 'content_id'])
    test_df['mean_diff1'] = test_df['mean_user_accuracy'] - test_df['mean_user_content_accuracy']
    test_df['mean_diff2'] = test_df['mean_accuracy'] - test_df['mean_user_content_accuracy']
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value = False).astype(bool)
    test_df = test_df.loc[test_df['content_type_id'] == 0].reset_index(drop=True)
    test_df.fillna(value = 0.5, inplace = True)
    test_df["prior_question_had_explanation_enc"] = le.fit_transform(test_df["prior_question_had_explanation"])

    for model in models:
        y_pred = model.predict_proba(test_df[columns], num_iteration=model.best_iteration_)[:, 1]
        y_preds.append(y_pred)

    y_preds = sum(y_preds) / len(y_preds)
    test_df['answered_correctly'] = y_preds
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

## Reference Link
[https://www.kaggle.com/vikassingh1996/riiid-eda-xgb-and-feature-importance](http://)